In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=fd914ff15f0b9cf2492c84d5e3921cb40d4cdff7ddb85b509ddf85ae0b929ca1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').appName('Scholze').getOrCreate()
sc = spark.sparkContext

sc.addFile('/content/drive/MyDrive/BIGDATA/transactions_amostra.csv')

rdd = sc.textFile("file://" + SparkFiles.get('transactions_amostra.csv'))

palavras = rdd.map(lambda x: x.split(";")).filter(lambda x: x[1].isnumeric()) 


In [ ]:
#Questão 1
pais = palavras.filter(lambda x: x[0]=='Brazil')
pais.take(10)
pais.countByKey().items()

dict_items([('Brazil', 54762)])

In [ ]:
#Questão 2
anos = palavras.map(lambda x: (x[1], 1))
anos.countByKey()

defaultdict(int,
            {'1988': 8568,
             '1989': 16916,
             '1990': 18188,
             '1991': 22652,
             '1992': 32860,
             '1993': 42031,
             '1994': 54143,
             '1995': 63875,
             '1996': 70390,
             '1997': 77876,
             '1998': 81117,
             '1999': 86191,
             '2000': 97572,
             '2001': 97240,
             '2002': 97202,
             '2003': 101125,
             '2004': 103671,
             '2005': 105913,
             '2006': 109186,
             '2007': 105891,
             '2008': 102680,
             '2009': 105686,
             '2010': 105407,
             '2011': 105486,
             '2012': 102989,
             '2013': 104734,
             '2014': 103231,
             '2015': 101973,
             '2016': 82277})

In [ ]:
#Questao 3
result = palavras.map(lambda x: (x[1] + " " + x[4], 1))

result.countByKey()

defaultdict(int,
            {'1988 Export': 3542,
             '1988 Import': 4268,
             '1988 Re-Export': 758,
             '1989 Export': 7092,
             '1989 Import': 8845,
             '1989 Re-Export': 979,
             '1990 Export': 7508,
             '1990 Import': 9642,
             '1990 Re-Export': 1038,
             '1991 Export': 9011,
             '1991 Import': 12331,
             '1991 Re-Export': 1310,
             '1992 Export': 12507,
             '1992 Import': 18840,
             '1992 Re-Export': 1513,
             '1993 Export': 15599,
             '1993 Import': 24450,
             '1993 Re-Export': 1982,
             '1994 Export': 20624,
             '1994 Import': 31425,
             '1994 Re-Export': 2094,
             '1995 Export': 22891,
             '1995 Import': 38679,
             '1995 Re-Export': 2305,
             '1996 Export': 25472,
             '1996 Import': 42467,
             '1996 Re-Export': 2451,
             '1997 Export': 2

In [ ]:
#Questao 4
# .groupByKey().map(lambda x: (x[0],sum(x[1])/len(x[1]))).take(5)
palavras.map(lambda x: (x[1], (float(x[5]),1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1]).take(10)

[('2011', 37056771.709060915),
 ('1991', 12069165.192609925),
 ('2005', 17458172.20780263),
 ('1999', 9328194.404264946),
 ('2000', 12780250.522332227),
 ('2009', 25688655.388272807),
 ('2014', 40771480.2568802),
 ('2012', 36993817.061880395),
 ('2004', 15283774.17530457),
 ('2002', 11605774.994269665)]

In [ ]:
#5  
#The average price of commodities per unit type, year, and category 
#in the export flow in Brazil;
# .groupByKey().map(lambda x: (x[0], sum(x[1])/len(x[1]))).take(10)
palavras.filter(lambda x: x[0]=="Brazil" and x[4]=="Export")\
.map(lambda x: (x[1]+" "+ x[7]+ " "+x[9],(float(x[5]),1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1]).take(10)




[('2016 Weight in kilograms 62_articles_of_apparel_accessories_not_knit_or_crochet',
  244220.66666666666),
 ('2014 Weight in kilograms 40_rubber_and_articles_thereof', 4892977.92),
 ('2014 Weight in kilograms 58_special_woven_or_tufted_fabric_lace_tapestry_etc',
  4610716.3),
 ('2005 Weight in kilograms 20_vegetable_fruit_nut_etc_food_preparations',
  4404850.0),
 ('2014 Weight in kilograms 60_knitted_or_crocheted_fabric',
  3110808.285714286),
 ('1993 Weight in kilograms 39_plastics_and_articles_thereof',
  8011417.485714286),
 ('1990 Weight in kilograms 62_articles_of_apparel_accessories_not_knit_or_crochet',
  885587.6666666666),
 ('2000 Weight in kilograms 30_pharmaceutical_products', 14549887.0),
 ('2013 Weight in kilograms 02_meat_and_edible_meat_offal', 569188439.9090909),
 ('2000 Weight in kilograms 72_iron_and_steel', 11737204.325581396)]

In [ ]:
#6
#The maximum, minimum, and mean transaction price per unit type and year;
# .groupByKey().map(lambda x: (x[0],[max(x[1]),min(x[1]),sum(x[1])/len(x[1])])).collect()

palavras.map(lambda x: (x[7] + " "+x[1],(float(x[5]),float(x[5]),float(x[5]),1.0)))\
.reduceByKey(lambda x,y: (x[0] if x[0]>y[0] else y[0],\
                          x[1] if x[1]<y[1] else y[1],\
                           x[2]+y[2], x[3]+y[3]))\
                           .map(lambda x: (x[0], (x[1][0],x[1][1], x[1][2]/x[1][3]))).take(10)

# alternativa
# palavras.map(lambda x: (x[7] + " "+x[1],float(x[5]))).groupByKey().map(lambda x: (x[0],[max(x[1]),min(x[1]),sum(x[1])/len(x[1])])).collect()

[('Volume in litres 2008', (3829869298.0, 2.0, 38542120.40059568)),
 ('Number of items 2002', (12582020967.0, 1.0, 16623866.827907434)),
 ('Weight in kilograms 1989', (5450526322.0, 1.0, 10364929.182174472)),
 ('Number of items 1993', (12730875972.0, 6.0, 29663512.0132964)),
 ('Area in square metres 2008', (1389350118.0, 9.0, 8935352.723)),
 ('Volume in litres 1993', (7105830176.0, 3.0, 36107479.83391978)),
 ('Area in square metres 1996', (594472645.0, 31.0, 8351314.379190751)),
 ('Number of pairs 1992', (829395508.0, 6.0, 31690831.04901961)),
 ('Volume in cubic meters 2004', (2770633668.0, 10.0, 42774729.605536334)),
 ('Number of pairs 2010', (4959592644.0, 5.0, 33584526.91748004))]

In [ ]:
#7 The  most  commercialized  commodity  (summing  the  quantities)  in  2016,  per  flow type. 
palavras.filter(lambda x: x[1]=="2016")\
.map(lambda x: ((x[4],x[3]),float(x[8])))\
.groupByKey().map(lambda x: (x[0][0], (sum(x[1]),x[0][1])))\
.reduceByKey(lambda x,y: x if x>y else y).collect()

[('Re-Export',
  (1261968000.0, 'Safety razor blades, including blanks in strips')),
 ('Export',
  (699847369665.0, 'Ice, snow and potable water not sweetened or flavoure')),
 ('Re-Import',
  (38774873.0, 'Chem wood pulp, soda/sulphate, non-conifer, bleached')),
 ('Import',
  (1073469712878.0, 'Iron ore, concentrate, not iron pyrites,unagglomerate'))]